# 15. Multiple Regression

## Imports

In [1]:
import datetime
import random
import sys
from typing import Callable, TypeVar

import numpy as np
import plotly.graph_objects as go
import tqdm
from sklearn.datasets import make_regression

In [2]:
# Importing functions from another notebook with supress graph rendering

_print = print
globals()["print"] = lambda *args, **kwargs: None

_show = go.Figure.show
go.Figure.show = lambda *args, **kwargs: None

_stdout = sys.stdout
_stderr = sys.stderr


class SupressTqdm(object):
    def write(self, x):
        pass


sys.stdout = SupressTqdm()
sys.stderr = SupressTqdm()

%run -i ../04_linear_algebra/main.ipynb
%run -i ../05_statistics/main.ipynb
%run -i ../06_probability/main.ipynb
%run -i ../08_gradient_descent/main.ipynb
%run -i ../14_simple_linear_regression/main.ipynb

globals()["print"] = _print
go.Figure.show = _show
sys.stdout = _stdout
sys.stderr = _stderr

In [3]:
random.seed(0)

## Model

In [4]:
def predict(x: Vector, beta: Vector) -> float:
    return dot(x, beta)

## Fitting Model

In [5]:
def error(x: Vector, y: float, beta: Vector) -> float:
    return predict(x, beta) - y

In [6]:
def squared_error(x: Vector, y: float, beta: Vector) -> float:
    return error(x, y, beta)**2

In [7]:
x = [1, 2, 3]
y = 30
beta = [4, 4, 4]

assert error(x, y, beta) == -6
assert squared_error(x, y, beta) == 36

In [8]:
def sqerror_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    err = error(x, y, beta)
    return [2 * err * x_i for x_i in x]


assert sqerror_gradient(x, y, beta) == [-12, -24, -36]

In [9]:
def least_squares_fit(xs: list[Vector], ys: Vector, learning_rate: float = 0.001, num_steps: int = 1000, batch_size: int = 1) -> Vector:
    guess = [random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="[+] Least squares fitting: ", bar_format="{desc}|{bar:50}|"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start + batch_size]
            batch_ys = ys[start:start + batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess) for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

In [10]:
learning_rate = 0.001

In [11]:
X, y = make_regression(n_samples=100, n_features=3, noise=42, random_state=0)

In [12]:
X = np.insert(X, 0, 1.0, axis=1)

In [13]:
beta = least_squares_fit(X, y, learning_rate, 5000, 25)

[+] Least squares fitting: |██████████████████████████████████████████████████|


In [14]:
print(f"[+] Coefficients: {[round(b, 4) for b in beta]}")

[+] Coefficients: [np.float64(-10.0923), np.float64(41.4454), np.float64(13.9702), np.float64(58.0754)]


## Quality of Fitting

In [15]:
def multiple_r_squared(xs: list[Vector], ys: Vector, beta: Vector) -> float:
    sum_of_squared_errors = sum(error(x, y, beta)**2 for x, y in zip(xs, ys))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(ys)

In [16]:
print(f"[+] R2-score: {multiple_r_squared(X, y, beta):.4f}")

[+] R2-score: 0.7609


## Samples Reproduction

In [17]:
x = TypeVar("X")
stat = TypeVar("Stat")

In [18]:
def bootstrap_sample(data: list[x]) -> list[x]:
    return [random.choice(data) for _ in data]

In [19]:
def bootstrap_statistic(data: list[x], stats_fn: Callable[[list[x]], stat], num_samples: int) -> list[stat]:
    return [stats_fn(bootstrap_sample(data)) for _ in range(101)]

In [20]:
close_to_100 = [99.5 + random.random() for _ in range(101)]

In [21]:
far_from_100 = ([99.5 + random.random()] + [random.random() for _ in range(50)] + [200 + random.random() for _ in range(50)])

In [22]:
medians_close = bootstrap_statistic(close_to_100, median, 100)

assert standard_deviation(medians_close) < 1

In [23]:
medians_far = bootstrap_statistic(far_from_100, median, 100)

assert standard_deviation(medians_far) > 90

## Standard Errors of Regression Coefficients

In [24]:
def estimate_sample_beta(pairs: list[tuple[Vector, float]]) -> Vector:
    x_sample = [x for x, _ in pairs]
    y_sapmle = [y for _, y in pairs]
    beta = least_squares_fit(x_sample, y_sapmle, learning_rate, 5000, 25)
    print(f"[+] Bootstrap selection: {[round(b, 4) for b in beta]}")
    return beta

In [25]:
bootstrap_betas = bootstrap_statistic(list(zip(X, y)), estimate_sample_beta, 100)

[+] Least squares fitting: |██████████████████████████████████████████████████|


[+] Bootstrap selection: [np.float64(-2.7188), np.float64(41.4377), np.float64(13.4025), np.float64(59.0156)]


[+] Least squares fitting: |██████████████████████████████████████████████████|


[+] Bootstrap selection: [np.float64(-7.7463), np.float64(42.3037), np.float64(15.6438), np.float64(56.4657)]


[+] Least squares fitting: |██████████████████████████████████████████████████|


...

[+] Bootstrap selection: [np.float64(-5.4053), np.float64(36.3863), np.float64(17.0522), np.float64(53.4014)]


[+] Least squares fitting: |██████████████████████████████████████████████████|


[+] Bootstrap selection: [np.float64(-13.0803), np.float64(39.3814), np.float64(11.421), np.float64(58.3909)]


[+] Least squares fitting: |██████████████████████████████████████████████████|

[+] Bootstrap selection: [np.float64(-9.714), np.float64(36.0215), np.float64(17.4381), np.float64(57.2069)]


In [26]:
bootstrap_standard_errors = [standard_deviation([beta[i] for beta in bootstrap_betas]) for i in range(4)]

In [27]:
print(f"[+] Standard deviations: {[round(error, 4) for error in bootstrap_standard_errors]}")

[+] Standard deviations: [4.1829, 3.9911, 4.9428, 3.3145]


In [28]:
def p_value(beta_hat_j: float, sigma_hat_j: float) -> float:
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

In [29]:
print(f"[+] P-value: {p_value(0, 4):.4f}")

[+] P-value: 1.0000


## Regularization

#### Ridge

In [30]:
def ridge_penalty(beta: Vector, alpha: float) -> float:
    return alpha * dot(beta[1:], beta[1:])

In [31]:
def squared_error_ridge(x: Vector, y: float, beta: Vector, alpha: float) -> float:
    return error(x, y, beta)**2 + ridge_penalty(beta, alpha)

In [32]:
def ridge_penalty_gradient(beta: Vector, alpha: float) -> Vector:
    return [0.0] + [2 * alpha * beta_j for beta_j in beta[1:]]

In [33]:
def sqerror_ridge_gradient(x: Vector, y: Vector, beta: Vector, alpha: float) -> Vector:
    return add(sqerror_gradient(x, y, beta), ridge_penalty_gradient(beta, alpha))

In [34]:
def least_squares_fit_ridge(xs: list[Vector], ys: Vector, alpha: float, learning_rate: float, num_steps: int, batch_size: int = 1) -> Vector:
    guess = [random.random() for _ in xs[0]]

    for i in range(num_steps):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start + batch_size]
            batch_ys = ys[start:start + batch_size]

            gradient = vector_mean([sqerror_ridge_gradient(x, y, guess, alpha) for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

In [35]:
beta_0 = least_squares_fit_ridge(X, y, 0.0, learning_rate, 5000, 25)

print(f"[+] Ridge coefficients with alpha 0: {[round(b, 4) for b in beta_0]}")

[+] Ridge coefficients with alpha 0: [np.float64(-10.0923), np.float64(41.4454), np.float64(13.9702), np.float64(58.0754)]


In [36]:
beta_0_1 = least_squares_fit_ridge(X, y, 0.1, learning_rate, 5000, 25)

print(f"[+] Ridge coefficients with alpha 0.1: {[round(b, 4) for b in beta_0_1]}")

[+] Ridge coefficients with alpha 0.1: [np.float64(-9.2208), np.float64(37.1664), np.float64(13.1095), np.float64(52.8417)]


In [37]:
beta_1 = least_squares_fit_ridge(X, y, 1, learning_rate, 5000, 25)

print(f"[+] Ridge coefficients with alpha 1: {[round(b, 4) for b in beta_1]}")

[+] Ridge coefficients with alpha 1: [np.float64(-5.5593), np.float64(19.1888), np.float64(8.2177), np.float64(29.2771)]


In [38]:
beta_10 = least_squares_fit_ridge(X, y, 10, learning_rate, 5000, 25)

print(f"[+] Ridge coefficients with alpha 10: {[round(b, 4) for b in beta_10]}")

[+] Ridge coefficients with alpha 10: [np.float64(-2.2852), np.float64(3.2667), np.float64(1.6908), np.float64(5.3794)]


#### Lasso

In [39]:
def lasso_penalty(beta: Vector, alpha: float) -> float:
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])